In [1]:
import reflex as rx
from reflex_experiment.helpers import create_camel_model
from pydantic.v1 import Field

# Original model with snake_case
class MyElement(rx.Base):
    client_height: float
    scroll_width: float
    content_editable: str
    async_: bool = Field(alias='async')


# Create camelCase version
MyElementNative = create_camel_model(MyElement)
MyElementNative.__fields__

# Now has fields: clientHeight, scrollWidth, contentEditable

{'clientHeight': ModelField(name='clientHeight', type=float, required=True),
 'scrollWidth': ModelField(name='scrollWidth', type=float, required=True),
 'contentEditable': ModelField(name='contentEditable', type=str, required=True),
 'async_': ModelField(name='async_', type=bool, required=True, alias='async')}